## Imports

In [1]:
import pandas as pd
import bz2
import json
import math
import datetime

## Datasets

In [2]:
DATA_FOLDER = '../data/'
SPEAKER_ATTRIBUTES_DATA = DATA_FOLDER+"speaker_attributes.parquet"
WIKIDATA_LABELS = DATA_FOLDER + "wikidata_labels_descriptions_quotebank.csv.bz2"

In [3]:
#Read the wikilabels dataframe
wiki_labels = pd.read_csv(WIKIDATA_LABELS, compression='bz2', index_col='QID')

In [4]:
#Read the attribute dataframe
attributes = pd.read_parquet(SPEAKER_ATTRIBUTES_DATA)

## Keep only the attributes of interest and rename the column for the future merge.

In [5]:
attributes = attributes.get(['gender','date_of_birth','occupation','id', 'ethnic_group', 'academic_degree', 'religion'])
attributes.rename(columns={'id':'qids'}, inplace=True)

## Preprocessing functions

In [6]:
def merge_chunk_attributes(chunk):
        """
        Merge the chunk with the .parquet file with the wikidata attribute of the speaker.
        Before doing it, keep only the quotes with a single qid.
    
        Parameters:
        ----------
        chunk: dataframe
    
        Returns:
        ----------
        chunk: the dataframe with readable information
    
        """
        #Remove the quotes with speaker linked to many qids: we cannot know which of the homonym is the real speaker
        chunk.drop(chunk[chunk['qids'].map(len) > 1].index, inplace=True)
        #Keep only the first qids
        chunk['qids'] = chunk['qids'].apply(lambda x: x[0])
        #Merge the chunk with the parquet file
        chunk_merged = chunk.merge(attributes)
        return chunk_merged
    
def match_code_label(code):
    """
        Match the wikidata code 'Q...' with its readable information
    
        Parameters:
        ----------
        code: string
    
        Returns:
        ----------
        string : the readable information
    
    """
    if code == None:
        return None
    
    if code in wiki_labels.index:
        return wiki_labels.loc[code]['Label']
    else:
        return None

def render_info_readable(chunk):
    """
    Replace the wikidata code 'Q...' by readable information. Drop all rows containing a None.
    
    Parameters:
    ----------
    chunk: dataframe
    
    Returns:
    ----------
    chunk: the dataframe with readable information
    
    """
    for col in ['gender', 'occupation', 'ethnic_group', 'academic_degree', 'religion']:
        chunk[col] = chunk[col].apply(lambda x: [match_code_label(i) for i in x] if (x is not None) else (None))

    return chunk


def filter_gender(chunk):
        """
        Keeps only the first gender in the list of genders, and drop the quotations with unknown gender.
    
        Parameters:
        ----------
        chunk: dataframe
    
        Returns:
        ----------
        chunk: the dataframe with only one gender per row.
    
        """  
        #Remove the quotations for which the gender of the speaker in unknown
        chunk.dropna(axis = 0, subset = ['gender'], inplace = True)
        #Keep only the first gender in the list 
        chunk['gender'] = chunk['gender'].apply(lambda x: x[0])
        return chunk


    
def get_age(chunk, actual_year):
    """
    Replace the column 'date_of_birth' by a column age, with the age calculated at the time of the quotation.
    
    Parameters:
    ----------
    chunk: dataframe
    actual_year: year of the dataframe
    
    Returns:
    ----------
    chunk: the dataframe with the age of speakers reported
    
    """
    
    chunk.dropna(axis = 0, subset = ['date_of_birth'], inplace = True)
    chunk['date_of_birth'] = chunk['date_of_birth'].apply(lambda x: x[0]) 
    chunk['date_of_birth'] = chunk['date_of_birth'].apply(lambda x: int(int(actual_year) - float(x[1:5]))) 
    chunk.rename(columns={"date_of_birth": "age"}, inplace = True)
    return chunk

def preprocess_data(year):
    """
    Preprocess the data by:
    -merging the data with the attributes of the speakers
    -rendering the information readable 
    -removing unknown gender
    -calculating age and removing unknown ages

    The function stores the preprocess data in files named "chunk-i-year.json.bz2", with i=1,...n for n 
    the number of chunks.
    
    Parameters:
    ----------
    year: string year of the dataframe to process. Expected values: '2015', '2016', '2017-before', 
    '2017-after', '2018', '2019' or '2020'.
    
    """
    DATA_TO_PROCESS = DATA_FOLDER + f'quotes-{year}-reduced.json.bz2'
    i = 0
    with pd.read_json(DATA_TO_PROCESS, lines=True, compression='bz2', chunksize=1000000) as df_reader:
        for chunk in df_reader:
            i +=1
            chunk_merged = merge_chunk_attributes(chunk)
            chunk_readable = render_info_readable(chunk_merged)
            chunk_filtered = filter_gender(chunk_readable)
            if year == '2017-after' or year == '2017-before':
                chunk_aged = get_age(chunk_filtered, '2017')
            else:
                chunk_aged = get_age(chunk_filtered, year)
            if year == '2017-after' or year == '2017-before' or year == '2018':
                chunk_aged.drop(columns = ['date'], inplace = True)
            chunk_aged.to_json(DATA_FOLDER + f'chunk-{str(i)}-{str(year)}.json.bz2')

def create_sample_from_year(year):
    """
    Create a sample for the year by randomly sampling 33333 quotations in the first 3 chunks created by 
    the function preprocess_data. The sample results in 99999 quotations for the given year. This function 
    stores the created sample in a file named "sample-year.json.bz2".
    
    Parameters:
    ----------
    year: string. Year of the dataframe to process. Expected values: '2015', '2016', '2017-before', 
    '2017-after', '2018', '2019' or '2020'.
    
    """
    sample = []
    for k in range(3):
        df_chunk = pd.read_json(DATA_FOLDER + f'chunk-{str(k+1)}-{year}.json.bz2')
        sample.append(df_chunk.sample(math.floor(100000/3)))
    df_sample = pd.concat(sample, ignore_index=True)
    df_sample.to_json(DATA_FOLDER + f'sample-{year}.json.bz2')
    


## Preprocess all years

In [7]:
years = ['2015', '2016', '2017-before', '2017-after', '2018', '2019', '2020']

In [8]:
for year in years:
    preprocess_data(year)

## Create dataframe ready for observational data

At this stage, we realized that the attributes `religion` and `ethnic_group` were very rarely known. So we decided to drop them.

In [9]:
def create_observational_data(year, nb_chunk):
    chunks=[]
    for i in range(nb_chunk):
        chunk = pd.read_json(DATA_FOLDER+f'chunk-{str(i+1)}-{year}.json.bz2')
        chunk.drop(['religion', 'ethnic_group'], axis = 1, inplace = True)
        chunk.dropna(inplace = True)
        chunks.append(chunk)
        df_obs = pd.concat(chunks)
        df_obs.reset_index(drop=True, inplace=True)
        df_obs.to_json(DATA_FOLDER + f'observational-{year}.json.bz2')

In [10]:
years = ['2015', '2016', '2017-before', '2017-after', '2018', '2019', '2020']
nb_chunks = [10, 7, 9, 4, 13, 10, 3]

In [11]:
for year, nb_chunk in zip(years, nb_chunks):
    create_observational_data(year, nb_chunk)

Now that we have created one file for each year, let's concatenate it into a single dataframe that we will use for the observational study.

In [12]:
quotes_2015 = pd.read_json(DATA_FOLDER+f'observational-2015.json.bz2')
quotes_2016 = pd.read_json(DATA_FOLDER+f'observational-2016.json.bz2')
quotes_2017_before = pd.read_json(DATA_FOLDER+f'observational-2017-before.json.bz2')
quotes_2017_after = pd.read_json(DATA_FOLDER+f'observational-2017-after.json.bz2')
quotes_2018 = pd.read_json(DATA_FOLDER+f'observational-2018.json.bz2')
quotes_2019 = pd.read_json(DATA_FOLDER+f'observational-2019.json.bz2')
quotes_2020 = pd.read_json(DATA_FOLDER+f'observational-2018.json.bz2')

We add a column `label`: 0 if the quotation is said before #MeToo, 1 if the quotation is said after #MeToo.

In [14]:
quotes_2015['label'] = 0
quotes_2016['label'] = 0
quotes_2017_before['label'] = 0
quotes_2017_after['label'] = 1
quotes_2018['label'] = 1
quotes_2019['label'] = 1
quotes_2020['label'] = 1

We also add a column `year` to still be able to access the year of the quote if needed.

In [15]:
quotes_2015['year'] = 2015
quotes_2016['year'] = 2016
quotes_2017_before['year'] = 2017
quotes_2017_after['year'] = 2017
quotes_2018['year'] = 2018
quotes_2019['year'] = 2019
quotes_2020['year'] = 2020

Let's finally create our final dataframe!

In [16]:
quotes = pd.concat([quotes_2015, quotes_2016, quotes_2017_before, 
                    quotes_2017_after, quotes_2018, quotes_2019, quotes_2020])

We will remove all quotations when the gender is not `male` or `female`.

In [17]:
quotes.drop(quotes[(quotes['gender'] != 'female') & (quotes['gender'] != 'male')].index, inplace = True)

In [18]:
quotes.head(4)

,quotation,speaker,qids,numOccurrences,gender,age,occupation,academic_degree,label,year
0,I am convinced that this conflict won't be sol...,Angela Merkel,Q567,1,female,61,"[politician, physicist, statesperson, chemist]",[doctorate],0,2015
1,agreed that the E.U. should take further measu...,Angela Merkel,Q567,6,female,61,"[politician, physicist, statesperson, chemist]",[doctorate],0,2015
2,Germany will continue to do everything to supp...,Angela Merkel,Q567,1,female,61,"[politician, physicist, statesperson, chemist]",[doctorate],0,2015
3,There has already been voluntary debt forgiven...,Angela Merkel,Q567,1,female,61,"[politician, physicist, statesperson, chemist]",[doctorate],0,2015


In [19]:
quotes.reset_index(drop=True, inplace = True)